## CMF 1
Institut für Musikinformatik und Musikwissenschaft – Wintersemester 2025–26
### Woche 07 – Vorlesung

### Globale und lokale Alignments mithilfe der Library *minineedle*

In [1]:
from minineedle import needle, smith, core

In [2]:
sequence1 = 'some trail_snacks'
sequence2 = 'a snail_track'

# Globales Alignment mit dem Needleman-Wunsch-Algorithmus
alignment_NW = needle.NeedlemanWunsch(sequence1, sequence2)
# Festlegen der Match-, Mismatch- und Gap-Scores
alignment_NW.change_matrix(core.ScoreMatrix(match=1, miss=-1, gap=-2))
alignment_NW.align()
score_NW = alignment_NW.get_score()
print(alignment_NW)
print("Alignment score: %i" % score_NW)

# Lokales Alignment mit dem Smith-Waterman-Algorithmus
alignment_SW = smith.SmithWaterman(sequence1, sequence2)
# Festlegen der Match-, Mismatch- und Gap-Scores
alignment_SW.change_matrix(core.ScoreMatrix(match=1, miss=-2, gap=-1))
alignment_SW.align()
score_SW = alignment_SW.get_score()
print(alignment_SW)
print("Alignment score: %i" % score_SW)

Alignment of SEQUENCE 1 and SEQUENCE 2:
	some trail_snacks
	---a snail_track-

Alignment score: -5
Alignment of SEQUENCE 1 and SEQUENCE 2:
	ail_
	ail_

Alignment score: 4


### Werktitel-Suche mit dem Needleman-Wunsch-Algorithmus und dem Smith-Waterman-Algorithmus

In [3]:
from minineedle import needle, smith, core

Die Datei *piece_titles.txt* enthält eine Liste mit Werktiteln (von denen z. B. Audio-Aufnahmen in einem Datenset vorliegen könnten) mit den jeweiligen Namen der Komponisten.\
Ziel dieser kurzen Aufgabe ist es, alle Einträge zu erfassen, die einem Werk von Beethoven entsprechen.\
Dabei nutzen wir den Needleman-Wunsch-Algorithmus und/oder den Smith-Waterman-Algorithmus (mit geeigneten Match-, Mismatch- und Gap-Scores) 

In [4]:
with open('piece_titles.txt', 'r', encoding='utf-8') as file:
    # Liste, die die verschiedenen Zeilen der Datei enthält
    title_list = [line.strip() for line in file.readlines()]

In [5]:
title_list[0]

'Beethoven - Symphony 1'

In [6]:
reference = 'BEETHOVEN'

In [7]:
# Es ist sinnvoll, einen Threshold-Score festzulegen. Scores, die über diesem Threshold liegen, gelten als Werk von Beethoven.
threshold_score = 6

In [8]:
# Liste, die die Alignment-Scores für alle Werktitel in der Datei enthält
scores = []
# Indizes (in der Liste title_list), die einem Werk von Beethoven (wahrscheinlich) entsprechen
found_indices = []

for i, title in enumerate(title_list):

    # Umwandlung aller Titel in Großbereich zum besseren Vergleichen verschiedener Schreibweisen
    title_in_capital_letters = title.upper()

    # Anwendung des Smith-Waterman-Algorithums (lokales Alignment)
    alignment = smith.SmithWaterman(reference, title_in_capital_letters)
    # Festlegen der Match-, Mismatch- und Gap-Scores
    alignment.change_matrix(core.ScoreMatrix(match=1, miss=-2, gap=-1))
    alignment.align()

    score = alignment.get_score()
    scores.append((title, score))

    if score > threshold_score:
        found_indices.append(i)

In [9]:
found_indices

[0, 1, 4, 9, 12, 14, 17, 19, 20, 21, 24, 27, 28, 29, 32, 34, 35, 39, 41, 44]

In [10]:
for index in found_indices:
    print(title_list[index])

Beethoven - Symphony 1
beethoven, Sinfonie 1
Beetoven - Sinfonie 5
Première Symphonie de Beethoven
veethoven, Symphony 3
Symphony 8- Beethoven
Piano Sonata 13 (Beethoven)
Beethoven: 3. Sinfonie in Es-Dur, op. 55, Beiname: „Eroica“ (Heroische Sinfonie)
?? beETHOVEN ??
Beethoven 9
beethovenSymphony 1
Beethove 3. Sinfonie
beethoven op.55
Track 02 (Live Recording 1981, Remastered 2010), BEETHOVEN - Symphony 7 in A Major
Beethoven: SYMPH. 5
BeethovenStringQuartet10
klaviersonate 31, beethoven
Piano Sonata?, Beethoven
Beethoven, Symphonie8
Beethovne - Symphony 09


Aus welchem Grund wäre der Needleman-Wunsch-Algorithmus (als *globaler* Alignment-Algorithmus) weniger geeignet, um alle Beethoven-Einträge zu identifizieren? 

In [13]:
title19 = title_list[19]
print(title19)

alignment = needle.NeedlemanWunsch(reference, title19.upper())
alignment.change_matrix(core.ScoreMatrix(match=1, miss=-2, gap=-1))
alignment.align()
score = alignment.get_score()

print(alignment)
print("Alignment score: %i" % score)

Beethoven: 3. Sinfonie in Es-Dur, op. 55, Beiname: „Eroica“ (Heroische Sinfonie)
Alignment of SEQUENCE 1 and SEQUENCE 2:
	BEETHOV--------------------------------------------------------------E------N---
	BEETHOVEN: 3. SINFONIE IN ES-DUR, OP. 55, BEINAME: „EROICA“ (HEROISCHE SINFONIE)

Alignment score: -62


In [14]:
title22 = title_list[22]
print(title22)

alignment = needle.NeedlemanWunsch(reference, title22.upper())
alignment.change_matrix(core.ScoreMatrix(match=1, miss=-2, gap=-1))
alignment.align()
score = alignment.get_score()

print(alignment)
print("Alignment score: %i" % score)

Mozart - Sonate no.13
Alignment of SEQUENCE 1 and SEQUENCE 2:
	--BEET---HO--VE-N----
	MOZART - SONATE NO.13

Alignment score: -18


Fazit: Globale Alignments zwischen Sequenzen, die sehr unterschiedliche Längen haben, sind oft wenig aussagekräftig.\
Hier: Sehr niedriger Score für einen langen Werktitel, in dem der Komponist Beethoven vorkommt; wesentlich höherer Score für einen kurzen Werktitel, in dem der Name Beethoven nicht vorkommt.